# Собираем первую нейросеть!

In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers as L
from tensorflow.keras import optimizers as opt
from tensorflow.keras import Sequential, Model

print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.0


In [2]:
from sklearn.datasets import load_boston
import pandas as pd

boston_dataset = load_boston()

df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target

df.head(n=10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.loc[:, df.columns != 'MEDV']
y = df.loc[:, df.columns == 'MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape

(354, 13)

In [4]:
y_train.shape

(354, 1)

In [5]:
model = Sequential(name='Archibald')

model.add(L.Dense(128, input_shape=(13, ), activation='relu', name='dense_1'))
model.add(L.Dense(64, activation='relu', name='dense_2'))
model.add(L.Dense(1, activation='linear', name='dense_output'))

optimizer = opt.SGD(lr=1e-3)

model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
model.summary()

Model: "Archibald"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1792      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
11/11 [==============================] - 1s 45ms/step - loss: 561.6711 - mae: 21.7741 - val_loss: 359.1847 - val_mae: 17.1094
Epoch 2/100
11/11 [==============================] - 0s 5ms/step - loss: 223.5201 - mae: 12.3397 - val_loss: 63.6231 - val_mae: 6.4401
Epoch 3/100
11/11 [==============================] - 0s 5ms/step - loss: 53.5481 - mae: 5.3334 - val_loss: 28.1164 - val_mae: 4.2640
Epoch 4/100
11/11 [==============================] - 0s 5ms/step - loss: 25.1683 - mae: 3.7985 - val_loss: 19.3312 - val_mae: 3.2663
Epoch 5/100
11/11 [==============================] - 0s 5ms/step - loss: 21.7559 - mae: 3.3646 - val_loss: 16.0986 - val_mae: 2.9406
Epoch 6/100
11/11 [==============================] - 0s 5ms/step - loss: 17.2228 - mae: 3.1581 - val_loss: 14.4181 - val_mae: 3.0610
Epoch 7/100
11/11 [==============================] - 0s 4ms/step - loss: 18.6072 - mae: 3.1503 - val_loss: 14.7170 - val_mae: 2.6813
Epoch 8/100
11/11 [==============================] - 0s 5ms/st

In [7]:
history.history.keys()

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])

In [8]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()

In [9]:
model.layers[-1].weights[1]

<tf.Variable 'dense_output/bias:0' shape=(1,) dtype=float32, numpy=array([0.6751438], dtype=float32)>

In [10]:
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 2ms/step - loss: 16.1091 - mae: 2.6637


[16.109121322631836, 2.663715124130249]

In [11]:
model.predict(X_test).T[0]

array([18.029684 , 26.706326 , 42.51579  , 20.86211  , 28.804913 ,
       43.895737 , 24.043266 ,  9.155586 , 19.211794 , 31.430984 ,
       19.43953  , 19.282705 , 13.843552 , 29.217299 , 19.019192 ,
       17.428844 , 20.46085  , 37.51542  , 16.917141 , 17.691673 ,
       11.963457 , 28.629955 , 33.604053 , 39.484913 , 37.65272  ,
       21.870033 , 13.5245695, 19.030415 , 18.823854 , 21.430077 ,
       24.585447 , 31.344624 , 12.344433 , 20.5318   , 21.037605 ,
       32.675304 , 24.943874 , 12.589388 , 15.2570305, 41.793045 ,
       28.333038 , 17.669468 , 25.355562 , 44.573593 , 21.364874 ,
       23.806625 , 18.1876   , 21.189388 , 15.40742  , 21.102707 ,
       32.061443 , 24.149057 , 17.867651 , 11.508037 , 21.396023 ,
       14.265347 , 13.463394 , 10.854004 , 33.532665 , 14.354166 ,
       17.774672 , 20.410408 , 16.925991 , 17.74041  , 21.910824 ,
       21.352621 , 22.582146 , 23.425121 , 24.5578   , 25.387627 ,
       20.135973 , 24.648123 , 15.273218 , 25.551014 , 15.4534

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)
mean_squared_error(y_test, y_pred_lr), mean_absolute_error(y_test, y_pred_lr)

(28.405854810508238, 3.6913626771162584)